# testing gpu

In [1]:
# import sys
# print("Python executable:", sys.executable)
# print("Environment path:", sys.prefix)

In [2]:
# import tensorflow as tf
# print("TensorFlow version:", tf.__version__)
# print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [3]:
# import tensorflow as tf
# tf.debugging.set_log_device_placement(True)

# # Example operation
# with tf.device('/GPU:0'):
#     a = tf.constant([[1.0, 2.0, 3.0]])
#     b = tf.reduce_sum(a)
#     print("Result:", b)

# testing model

In [11]:
from tensorflow.keras.models import load_model

try:
    model = load_model("models/first.weights.h5")
    config = model.get_config()  # Attempt to retrieve model configuration
    print("Model configuration exists:", config)
except Exception as e:
    print("Error:", str(e))

Error: No model config found in the file at models/first.weights.h5.


# testing request

getting dictionary to match the output number from the model to the name of the car

In [4]:
retrieved_dict = {}
with open('data/label_to_number.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(': ', 1)
        retrieved_dict[key] = value

Getting the path for the training images

In [5]:
import os
output_file = "data/train_test_split.txt"
path_list = []

if os.path.exists(output_file): # if path exist
    with open(output_file, "r") as f:
        for line in f:
            split_type, path = line.strip().split("\t")
            if int(split_type) == 2:
                path_list.append(path)

print(path_list[:5])

['data/cars196/test\\AM General Hummer SUV 2000\\000046.jpg', 'data/cars196/test\\AM General Hummer SUV 2000\\000047.jpg', 'data/cars196/test\\AM General Hummer SUV 2000\\000048.jpg', 'data/cars196/test\\AM General Hummer SUV 2000\\000049.jpg', 'data/cars196/test\\AM General Hummer SUV 2000\\000050.jpg']


Getting label

In [6]:
import re
label_list = []

# Iterate through each entry in the list
for path in path_list:
    # Use regex to find text between backslashes
    match = re.search(r'\\(.*?)\\', path)
    if match:
        car_name = match.group(1)
        label_list.append(car_name)

print(label_list[:5])

['AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000']


In [7]:
# Function to load and preprocess images and label
def load_and_preprocess_image(image_path, label, target_size=(224, 224)):
    image = tf.io.read_file(image_path) # Load
    image = tf.image.decode_jpeg(image, channels=3) # Decode
    image = tf.image.resize(image, target_size) # Resize
    image = image / 255.0 # Normalize
    
    # convert string label to int
    label_number = tf.py_function(func=lookup_label, inp=[label], Tout=tf.int32)
    # Make sure label is scalar
    label_number = tf.reshape(label_number, ())
    
    return image, label_number

def lookup_label(label):
    # Convert Tensor label to Python string
    label_str = label.numpy().decode('utf-8')
    # Look up the label in the dictionary
    return label_to_number[label_str]

In [8]:
import tensorflow as tf
size = (224, 224)
ds_test = tf.data.Dataset.from_tensor_slices((path_list, label_list))
ds_test = ds_test.map(lambda image_path, label: load_and_preprocess_image(image_path, label, size))

In [10]:
import requests

url = "http://localhost:5000/predict"

for img in path_list:
    input_data = {'input': img}
    try:
        response_json = response.json()  # Attempt to parse JSON
        print(response_json)
    except requests.JSONDecodeError:
        print("Response is not JSON:", response.text)

Response is not JSON: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Response is not JSON: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Response is not JSON: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Response is not JSON: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The serv